In [97]:
import requests
from bs4 import BeautifulSoup
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}

In [103]:
response = requests.get("http://cn.chinadaily.com.cn/", headers=headers)
response.encoding = 'utf-8'

In [104]:
response.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n\n<html xmlns="http://www.w3.org/1999/xhtml">\n  <head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" />\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" />\n    <title>中国日报网-传播中国，影响世界</title>\n    <meta content="中国日报网,中国日报,国内新闻,国际快讯,评论,财经,体育,军事,图片,文化娱乐,时尚生活,视频" name="keywords" />\n    <meta content="中国日报网是中国最优质的新闻发布平台,热点实时新闻发布,提供国内新闻,国际快讯,评论,财经,体育,军事,图片,文化娱乐,时尚生活,视频及互动等综合新闻资讯!" name="description" />\n    <link rel="canonical" href="https://cn.chinadaily.com.cn" />\n    <link rel="stylesheet" charset="utf-8" href="//img3.chinadaily.com.cn/static/2018cn/css/publicstyle.css?v=15" /><!--头部和尾部右侧二维码-->\n    <link rel="stylesheet" charset="utf-8" href="//img3.chinadaily.com.cn/static/2018cn/css/

In [59]:
soup = BeautifulSoup(response.text,'html.parser')
links = soup.find_all("a", attrs={"shape":"rect"},  href=True)
links

[<a href="//www.chinadaily.com.cn/mobile_cn/index.html" shape="rect" target="_blank">移动新媒体</a>,
 <a href="//www.chinadaily.com.cn/" shape="rect" target="_blank"><img src="//img3.chinadaily.com.cn/static/2018cn/img/p-3.png"/></a>,
 <a href="//cn.chinadaily.com.cn" shape="rect" target="_blank">首页</a>,
 <a href="//china.chinadaily.com.cn" shape="rect" target="_blank">时政</a>,
 <a href="//china.chinadaily.com.cn/5bd5639ca3101a87ca8ff636" shape="rect" target="_blank">时政要闻</a>,
 <a href="//china.chinadaily.com.cn/5bd5639ca3101a87ca8ff634" shape="rect" target="_blank">评论频道</a>,
 <a href="//china.chinadaily.com.cn/hexin" shape="rect" target="_blank">核心解读</a>,
 <a href="//cn.chinadaily.com.cn/5b753f9fa310030f813cf408/5b940cbaa310030f813ed4d7" shape="rect" target="_blank">老外谈</a>,
 <a href="//china.chinadaily.com.cn/theory" shape="rect" target="_blank">理论频道</a>,
 <a href="//china.chinadaily.com.cn/5bd5639ca3101a87ca8ff634/5bd5669ba3101a87ca8ff666" shape="rect" target="_blank">海外观察</a>,
 <a href="

In [81]:
urls = ["http:"+ link['href'] for link in links if 'http' not in link['href']]

In [144]:
import sys
!{sys.executable} -m pip install validators
import validators

In [153]:
urls = [url for url in urls if validators.url(url)]

In [154]:
urls

['http://www.chinadaily.com.cn/mobile_cn/index.html',
 'http://www.chinadaily.com.cn/',
 'http://cn.chinadaily.com.cn',
 'http://china.chinadaily.com.cn',
 'http://china.chinadaily.com.cn/5bd5639ca3101a87ca8ff636',
 'http://china.chinadaily.com.cn/5bd5639ca3101a87ca8ff634',
 'http://china.chinadaily.com.cn/hexin',
 'http://cn.chinadaily.com.cn/5b753f9fa310030f813cf408/5b940cbaa310030f813ed4d7',
 'http://china.chinadaily.com.cn/theory',
 'http://china.chinadaily.com.cn/5bd5639ca3101a87ca8ff634/5bd5669ba3101a87ca8ff666',
 'http://world.chinadaily.com.cn',
 'http://world.chinadaily.com.cn/5bd55927a3101a87ca8ff618',
 'http://cnews.chinadaily.com.cn',
 'http://cn.chinadaily.com.cn/5b753f9fa310030f813cf408/5b9408aca310030f813ed4d4',
 'http://cn.chinadaily.com.cn/5b753f9fa310030f813cf408/5bd54dd6a3101a87ca8ff5f8/5bd54e59a3101a87ca8ff606',
 'http://world.chinadaily.com.cn/5bd55927a3101a87ca8ff616',
 'http://world.chinadaily.com.cn/5bd97038a3101a87ca904233',
 'http://caijing.chinadaily.com.cn',

In [178]:

def get_chinese_news(start_url, limit):
    urls = [start_url]
    processed_urls = []
    chinese_news_file = open('chinese_news_bfs.text', 'a+')
    counts = 0
   
    while True:
        if counts == limit:
            print("Processed {} pages, target completed".format(counts))
            return
        url = urls.pop(0)
        if url in processed_urls:
            print("Already processed: " + url)
        else:
            response = requests.get(url, headers=headers)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text)
            # find news text on this page
            ps = soup.find_all('p')
            news_text = ''.join([p.string for p in ps if p.string is not None])
            if len(news_text) < 100:
                continue
            chinese_news_file.write(news_text+"\n")
            # find urls on this page
            links = soup.find_all("a", attrs={"shape":"rect"},  href=True)
            new_urls = ["http:"+ link['href'] for link in links if 'http' not in link['href'] and 'chinadaily.com.cn' in link['href']]
            new_urls = [url for url in new_urls if validators.url(url) and url not in processed_urls]
            # add new urls 
            urls += new_urls
            processed_urls.append(url)
            counts += 1


In [179]:
get_chinese_news("http://china.chinadaily.com.cn", 2000)

Already processed: http://china.chinadaily.com.cn/hexin
Already processed: http://china.chinadaily.com.cn/5bd5639ca3101a87ca8ff634
Already processed: http://china.chinadaily.com.cn/theory
Already processed: http://china.chinadaily.com.cn/5bd5639ca3101a87ca8ff62e
Already processed: http://china.chinadaily.com.cn/5bd8126ba3101a87ca90094b
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d57a09aa31099ab995da101.html
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d580858a31099ab995da14f.html
Already processed: http://china.chinadaily.com.cn/5bd5639ca3101a87ca8ff634
Already processed: http://china.chinadaily.com.cn/a/201908/17/WS5d57ae2ea31099ab995da113.html
Already processed: http://china.chinadaily.com.cn/a/201908/17/WS5d57ae13a31099ab995da110.html
Already processed: http://china.chinadaily.com.cn/a/201907/27/WS5d3c239da3106bab40a02d12.html
Already processed: http://china.chinadaily.com.cn/a/201907/13/WS5d29af83a310a6dd41e85bdd.html
Already processed: http://chi

Already processed: http://china.chinadaily.com.cn/a/201908/16/WS5d5651e2a31099ab995d9f2c.html
Already processed: http://china.chinadaily.com.cn/a/201908/14/WS5d536cc8a31099ab995d94f2.html
Already processed: http://china.chinadaily.com.cn/a/201908/13/WS5d521a3ba31099ab995d90ab.html
Already processed: http://china.chinadaily.com.cn/a/201908/13/WS5d521106a31099ab995d906b.html
Already processed: http://china.chinadaily.com.cn/a/201908/12/WS5d50c059a31099ab995d8bdb.html
Already processed: http://china.chinadaily.com.cn/a/201908/12/WS5d50c058a31099ab995d8bda.html
Already processed: http://china.chinadaily.com.cn/a/201908/09/WS5d4d287fa31099ab995d879d.html
Already processed: http://china.chinadaily.com.cn/a/201908/08/WS5d4bdff3a31099ab995d8335.html
Already processed: http://china.chinadaily.com.cn/a/201908/08/WS5d4bdff3a31099ab995d8335.html
Already processed: http://china.chinadaily.com.cn/a/201908/07/WS5d4a7faca31099ab995d7ef0.html
Already processed: http://china.chinadaily.com.cn/a/201908/0

Already processed: http://cn.chinadaily.com.cn/a/201907/15/WS5d2bc06da310a6dd41e85d5a.html
Already processed: http://cn.chinadaily.com.cn/a/201907/15/WS5d2bc06da310a6dd41e85d5a.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59fb76a31099ab995da2eb.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59fa78a31099ab995da2e2.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59f527a31099ab995da281.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59f308a31099ab995da270.html
Already processed: http://china.chinadaily.com.cn/a/201908/18/WS5d594a54a31099ab995da225.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d59d22fa31099ab995da235.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d591747a31099ab995da1f4.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d591744a31099ab995da1f3.html
Already processed: http://cn.chinadaily.com.cn
Already processed: http://cn.chinadaily.

Already processed: http://cn.chinadaily.com.cn/5b753f9fa310030f813cf408/5bd54ba2a3101a87ca8ff5ec
Already processed: http://cn.chinadaily.com.cn/a/201907/15/WS5d2bc06da310a6dd41e85d5a.html
Already processed: http://cn.chinadaily.com.cn/a/201907/15/WS5d2bc06da310a6dd41e85d5a.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59fb76a31099ab995da2eb.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59fa78a31099ab995da2e2.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59f527a31099ab995da281.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59f308a31099ab995da270.html
Already processed: http://china.chinadaily.com.cn/a/201908/18/WS5d594a54a31099ab995da225.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d59d22fa31099ab995da235.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d591747a31099ab995da1f4.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d591744a31099ab995d

Already processed: http://cn.chinadaily.com.cn/a/201907/15/WS5d2bc06da310a6dd41e85d5a.html
Already processed: http://cn.chinadaily.com.cn/a/201907/15/WS5d2bc06da310a6dd41e85d5a.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59fb76a31099ab995da2eb.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59fa78a31099ab995da2e2.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59f527a31099ab995da281.html
Already processed: http://cn.chinadaily.com.cn/a/201908/19/WS5d59f308a31099ab995da270.html
Already processed: http://china.chinadaily.com.cn/a/201908/18/WS5d594a54a31099ab995da225.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d59d22fa31099ab995da235.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d591747a31099ab995da1f4.html
Already processed: http://cn.chinadaily.com.cn/a/201908/18/WS5d591744a31099ab995da1f3.html
Already processed: http://cn.chinadaily.com.cn
Already processed: http://cn.chinadaily.

...

Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d424016a3106bab40a039d2.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d424016a3106bab40a039d2.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d424015a3106bab40a039d1.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d424015a3106bab40a039d1.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d424d2fa3106bab40a03a6e.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d424d2ba3106bab40a03a6d.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d424d2aa3106bab40a03a6c.html
Already processed: http://caijing.chinadaily.com.cn/a/201908/01/WS5d4249e1a3106bab40a03a63.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d424678a3106bab40a03a41.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d423f0ca3106bab40a039c2.html
Already processed: http://cn.chinadaily.com.cn/a/201908/01/WS5d423bf6a3106bab40a03957

Already processed: http://language.chinadaily.com.cn/a/201908/14/WS5d535b56a310cf3e35565987.html
Already processed: http://language.chinadaily.com.cn/a/201908/14/WS5d535b56a310cf3e35565987.html
Already processed: http://language.chinadaily.com.cn/a/201908/13/WS5d524d70a310cf3e35565778.html
Already processed: http://language.chinadaily.com.cn/a/201908/13/WS5d524d70a310cf3e35565778.html
Already processed: http://www.chinadaily.com.cn/mobile_cn/web.html
Already processed: http://www.chinadaily.com.cn/mobile_cn/web.html
Already processed: http://language.chinadaily.com.cn/a/201908/15/WS5d54aceba310cf3e35565d46.html
Already processed: http://language.chinadaily.com.cn/a/201908/15/WS5d54aceba310cf3e35565d46.html
Already processed: http://language.chinadaily.com.cn/a/201908/14/WS5d535b34a310cf3e35565984.html
Already processed: http://language.chinadaily.com.cn/a/201908/14/WS5d535b34a310cf3e35565984.html
Already processed: http://language.chinadaily.com.cn/a/201908/13/WS5d520c20a310cf3e3556560

Already processed: http://world.chinadaily.com.cn
Already processed: http://cn.chinadaily.com.cn/a/201908/16/WS5d55fc62a31099ab995d9cda.html
Already processed: http://cn.chinadaily.com.cn/a/201908/16/WS5d55fc62a31099ab995d9cda.html
Already processed: http://world.chinadaily.com.cn/a/201908/17/WS5d57ad7fa31099ab995da10a.html
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d579425a31099ab995da0e3.html
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d579355a31099ab995da0e2.html
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d578ca5a31099ab995da0db.html
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d576050a31099ab995da096.html
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d576017a31099ab995da095.html
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d574c9aa31099ab995da07e.html
Already processed: http://cn.chinadaily.com.cn/a/201908/17/WS5d573e1ea31099ab995da079.html
Already processed: http://cn.chinadai

Already processed: http://www.chinadaily.com.cn/a/201908/19/WS5d598353a310cf3e355665fd.html
Already processed: http://www.chinadaily.com.cn/a/201908/19/WS5d59fb87a310cf3e35566735.html
Already processed: http://www.chinadaily.com.cn/a/201908/19/WS5d59fb87a310cf3e35566735.html
Already processed: http://www.chinadaily.com.cn/a/201908/19/WS5d59f66aa310cf3e355666fa.html
Already processed: http://www.chinadaily.com.cn/a/201908/19/WS5d59f66aa310cf3e355666fa.html
Already processed: http://www.chinadaily.com.cn/a/201908/17/WS5d5788b0a310cf3e355664e8.html
Already processed: http://www.chinadaily.com.cn/a/201908/16/WS5d56446ba310cf3e355662d6.html
Already processed: http://www.chinadaily.com.cn/a/201908/17/WS5d57924ba310cf3e35566504.html
Already processed: http://www.chinadaily.com.cn/a/201908/17/WS5d578b5da310cf3e355664f6.html
Already processed: http://www.chinadaily.com.cn/a/201908/16/WS5d564a2ba310cf3e355662fb.html
Already processed: http://www.chinadaily.com.cn/a/201908/16/WS5d565414a310cf3e35

ConnectionError: HTTPConnectionPool(host='english.jinan.gov.cn', port=80): Max retries exceeded with url: /art/2019/8/15/art_29558_3186310.html (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fcf54c8dda0>: Failed to establish a new connection: [Errno 110] Connection timed out'))